In this file I start by implementing a compressed sensing reconstruction approach. I want to have a well accepted classical method 
as a baseline for my later deep learning models.

In [1]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"]= '0' #, this way I would choose GPU 3 to do the work

sys.path.append('../scripts')
sys.path.append('../models')

from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom # for compressing images / only for testing purposes to speed up NN training
from torch.utils.data import DataLoader, Subset
import torch.optim as optim
import torch.nn as nn
from data_preparation import *
from data_undersampling import *
#from Naive_CNN_3D_Residual import *
from output_statistics import *
import plotly.graph_objects as go

In [2]:
#### keep three middle rows, delete every other row after that next time step the opposite

ground_truth = np.load('Full_Rank_All.npy')

In [54]:
def estimate_noise_variance(noise_data):
    """
    Estimate the noise variance from a dataset that contains only noise.

    Parameters:
    - noise_data (numpy.ndarray): A complex-valued array of noise-only data.

    Returns:
    - variance_real (float): Estimated variance of the real part.
    - variance_imag (float): Estimated variance of the imaginary part.
    - total_variance (float): Average variance across real and imaginary components.
    """
    real_part = noise_data.real
    imag_part = noise_data.imag

    variance_real = np.var(real_part, ddof=1)  # Use ddof=1 for unbiased estimate
    variance_imag = np.var(imag_part, ddof=1)

    total_variance = (variance_real + variance_imag) / 2  # Mean variance

    return variance_real, variance_imag, total_variance

In [77]:
for m in range(0,6):
    noise = ground_truth[:,:,:,66:,:,m] # last 30 FID time steps are just noise
    noise_variance = []
    for i in range(noise.shape[-2]):
        for j in range(noise.shape[-1]):
            noise_variance.append(estimate_noise_variance(noise[:,:,:,i,j])[-1])
    averaged_noise_variance = np.mean(noise_variance)  
    print(averaged_noise_variance)

11772658.675
10966683.491666667
11630350.5125
10794213.091666667
11400789.608333332
11372949.8625
